In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mdtraj as md
import os
import math
import time
from sys import exit

In [2]:
np.__version__

'1.24.3'

In [2]:
residue_names = ['ALA', 'ARG', 'ASN','ASP', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'HYP', 'ILE', 'LEU', 'LYS'
    ,'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']

nucleotide_names = ['DA', 'DT', 'DG', 'DC']

In [3]:
phosphate_atoms = ['OP1', 'OP2', 'P']
sugar_atoms = ['C1\'', 'C2\'', 'C3\'', 'C4\'', 'C5\'', 'O3\'', 'O4\'', 'O5\'']

In [4]:
def remove_water(traj):
    """
    Removes water molecules from a mdtraj.Trajectory object.
    
    Parameters
    ----------
    traj : mdtraj.Trajectory
        The input trajectory.
    
    Returns
    -------
    mdtraj.Trajectory
        The output trajectory containing only non-water atoms.
    """
    # Select all non-water atoms
    non_water_atoms = traj.topology.select("not water")

    # Create a new trajectory containing only the non-water atoms
    traj_no_water = traj.atom_slice(non_water_atoms)

    return traj_no_water

def remove_hyrdogen(traj):
    """
    Removes hydrogen atoms from a mdtraj.Trajectory object.
    
    Parameters
    ----------
    traj : mdtraj.Trajectory
        The input trajectory.
    
    Returns
    -------
    mdtraj.Trajectory
        The output trajectory containing only non-hydrogen atoms.
    """
    # Select all non-hydrogen atoms
    non_hydrogen_atoms = traj.topology.select("not element H")

    # Create a new trajectory containing only the non-hydrogen atoms
    traj_no_hydrogen = traj.atom_slice(non_hydrogen_atoms)

    return traj_no_hydrogen

In [28]:
data_folder = './data/ssDNA_binding_proteins_complex'

In [29]:
file_name = '8DFA.pdb'
file_path = os.path.join(data_folder, file_name)

traj = md.load_pdb(file_path) 

/Users/marvinprakash/anaconda3/envs/bio/lib/python3.8/site-packages/mdtraj/formats/pdb/pdbfile.py:200: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


In [30]:
print(traj.n_atoms)
# Remove water
traj = remove_water(traj)

print(traj.n_atoms)
# Remove hydrogen
traj = remove_hyrdogen(traj)

print(traj.n_atoms)

# Remove ligands
# traj = remove_ligands(traj)

46813
46813
23790


In [33]:
n_atoms = traj.n_residues
bond_info = {'indices': np.array([[i,i+1] for i in range(n_atoms - 1)])}

In [56]:
top = traj.topology
ca_indices = top.select('name CA')

top_ca = top.subset(ca_indices)
top_ca

traj_ca = traj.atom_slice(ca_indices)
traj_ca

<mdtraj.Trajectory with 1 frames, 464 atoms, 464 residues, and unitcells at 0x15bd28430>

In [8]:
def get_chains(file_path:str):
    # Load the traj
    traj = md.load_pdb(file_path)
    # Remove water
    traj = remove_water(traj)
    top = traj.topology
    chains = {}
    for chain in top.chains:
        # print(list(chain.residues))
        first_residue = chain.residue(0).name
        if first_residue in residue_names:
            chain_type = 'protein'
        elif first_residue in nucleotide_names:
            chain_type = 'ssDNA'
        else:
            continue
        chains[chain.index] = {
            'residues': list(chain.residues),
            'type': chain_type,
            'residue_names': [x.name for x in chain.residues]
        }
    return chains
    

In [9]:
def remove_dimers(chains):
    for chain in chains.values():
        if chain['type'] == 'ssDNA':
            continue
        # Check if the chain's residue name list is a subarray of others
        for other_chain in chains.values():
            if chain == other_chain:
                continue
            if chain['residue_names'] in other_chain['residue_names']:
                chains.pop(chain)
                break
    return chains   

In [19]:
dir(traj.topology)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_atoms',
 '_bonds',
 '_chains',
 '_numAtoms',
 '_numResidues',
 '_residues',
 '_standardBonds',
 '_string_summary_basic',
 '_unique_pairs',
 '_unique_pairs_equal',
 '_unique_pairs_mutually_exclusive',
 'add_atom',
 'add_bond',
 'add_chain',
 'add_residue',
 'atom',
 'atoms',
 'atoms_by_name',
 'bonds',
 'chain',
 'chains',
 'copy',
 'create_disulfide_bonds',
 'create_standard_bonds',
 'delete_atom_by_index',
 'find_molecules',
 'from_dataframe',
 'from_openmm',
 'guess_anchor_molecules',
 'insert_atom',
 'join',
 'n_atoms',
 'n_bonds',
 'n_chains',
 'n_residues',
 'residue',
 'residues',
 's

In [20]:
chains = get_chains(file_path)
chains = remove_dimers(chains)
remaining_chain_indices = list(chains.keys())

top = traj.topology
for chain in top.chains:
    if 
    

# Remove chains not in the new list indices
new_topology = traj.topology.subset([atom.index for atom in traj.topology.atoms if atom.residue.chain.index not in remaining_chain_indices])

# Create a new trajectory with the updated topology
new_traj = md.Trajectory(xyz=traj.xyz, topology=new_topology)

filtered_data_folder = "./data/processed_pdbs"
# Save the new trajectory
new_traj.save_pdb(os.path.join(filtered_data_folder, file_name))


/Users/marvinprakash/anaconda3/envs/bio/lib/python3.8/site-packages/mdtraj/formats/pdb/pdbfile.py:200: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


ValueError: xyz must be shape (Any, 1480, 3). You supplied  (1, 46813, 3)

In [18]:
(list(chains.values()))[11]

{'residues': [DT16,
  DC17,
  DG18,
  DC19,
  DC20,
  DA21,
  DG22,
  DC23,
  DC24,
  DT25,
  DG26,
  DA27,
  DG28,
  DC29,
  DA30,
  DT31,
  DG32,
  DG33],
 'type': 'ssDNA',
 'residue_names': ['DT',
  'DC',
  'DG',
  'DC',
  'DC',
  'DA',
  'DG',
  'DC',
  'DC',
  'DT',
  'DG',
  'DA',
  'DG',
  'DC',
  'DA',
  'DT',
  'DG',
  'DG']}

In [35]:
chain1 = chains[0]['residues']
dir(chain1[0].atom(1))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'element',
 'index',
 'is_backbone',
 'is_sidechain',
 'n_bonds',
 'name',
 'residue',
 'segment_id',
 'serial']

In [17]:
atoms = [x.name for x in list(chain_temp['residues'][0].atoms)]
for atom in atoms:
    print(atom, end=" ")

P OP1 OP2 O5' C5' C4' O4' C3' O3' C2' C1' N1 C2 O2 N3 C4 O4 C5 C7 C6 H5' H5'' H4' H3' H2' H2'' H1' H3 H71 H72 H73 H6 

In [9]:
top.residues

<generator object Topology.residues at 0x1589cdd60>

In [72]:
for i in range(n_atoms - 1):
    bond_length = md.compute_distances(traj_ca, atom_pairs=[[i,i+1]])
    # Get the average distances
    bond_length = np.mean(bond_length)
    print(f'The distance between residue {i+1} and {i+2} is {bond_length:.3f} nm')

The distance between residue 1 and 2 is 0.384 nm
The distance between residue 2 and 3 is 0.387 nm
The distance between residue 3 and 4 is 0.383 nm
The distance between residue 4 and 5 is 0.381 nm
The distance between residue 5 and 6 is 0.380 nm
The distance between residue 6 and 7 is 0.374 nm
The distance between residue 7 and 8 is 0.378 nm
The distance between residue 8 and 9 is 0.376 nm
The distance between residue 9 and 10 is 0.374 nm
The distance between residue 10 and 11 is 0.374 nm
The distance between residue 11 and 12 is 0.376 nm
The distance between residue 12 and 13 is 0.378 nm
The distance between residue 13 and 14 is 0.377 nm
The distance between residue 14 and 15 is 0.377 nm
The distance between residue 15 and 16 is 0.379 nm
The distance between residue 16 and 17 is 0.377 nm
The distance between residue 17 and 18 is 0.379 nm
The distance between residue 18 and 19 is 0.381 nm
The distance between residue 19 and 20 is 0.382 nm
The distance between residue 20 and 21 is 0.380 

ValueError: atom_pairs must be between 0 and 464

In [57]:
for residue in top.residues:
    print(residue)
    sidechain_indices = residue.select('sidechain')
    for atom in residue.atoms:
        print(atom.is_sidechain)
        # Get indices of all sidechain atoms
        sidechain_indices = top.select('sidechain')

GLY1


AttributeError: 'Residue' object has no attribute 'select'

In [48]:
for chain in top.chains:
    
    



[<mdtraj.core.topology.Chain object at 0x15aeccac0>, <mdtraj.core.topology.Chain object at 0x15aecccd0>, <mdtraj.core.topology.Chain object at 0x15aee1520>]
[<mdtraj.core.topology.Chain object at 0x15aeccac0>, <mdtraj.core.topology.Chain object at 0x15aecccd0>, <mdtraj.core.topology.Chain object at 0x15aee1520>]
[<mdtraj.core.topology.Chain object at 0x15aeccac0>, <mdtraj.core.topology.Chain object at 0x15aecccd0>, <mdtraj.core.topology.Chain object at 0x15aee1520>]


In [38]:
for i in range(top.n_residues):
    for 
    print(f'Residue {i+1} is a {top.residue(i).name}')
    sidechain_residues = top.select(f'resid {i} sidechain')
    # Get COM of sidechain
    sidechain_com = md.compute_center_of_mass(traj.atom_slice(sidechain_residues))
    sidechain_com = np.mean(sidechain_com, axis=0)
    print(f'The center of mass of the sidechain is {sidechain_com}')
    # Get the distance between COM and alpha carbon
    alpha_carbon_index = top.select(f'resid {i} and name CA')

Residue 1 is a GLY
The center of mass of the sidechain is [3.53926015 0.93662286 2.01074283]
Residue 2 is a ALA
The center of mass of the sidechain is [3.44168052 1.25799412 2.06188182]
Residue 3 is a MET
The center of mass of the sidechain is [3.70873265 1.58557987 1.86859955]
Residue 4 is a GLY
The center of mass of the sidechain is [3.61928235 1.74221859 2.33701852]
Residue 5 is a THR
The center of mass of the sidechain is [3.46632533 1.48255537 2.52989899]
Residue 6 is a ASN
The center of mass of the sidechain is [3.79875747 1.37877504 2.74564533]
Residue 7 is a LEU
The center of mass of the sidechain is [3.79944788 1.62632238 3.15084283]
Residue 8 is a TYR
The center of mass of the sidechain is [4.42487586 1.62588955 3.37713205]
Residue 9 is a ILE
The center of mass of the sidechain is [4.06511791 1.5792651  3.691659  ]
Residue 10 is a ARG
The center of mass of the sidechain is [4.56653639 1.86175189 3.86585649]
Residue 11 is a GLY
The center of mass of the sidechain is [4.3799400

In [41]:
for atom in top.atoms:
    print(atom)

GLY1-N
GLY1-CA
GLY1-C
GLY1-O
GLY1-H
GLY1-H2
GLY1-H3
GLY1-HA2
GLY1-HA3
TYR2-N
TYR2-CA
TYR2-C
TYR2-O
TYR2-CB
TYR2-CG
TYR2-CD1
TYR2-CD2
TYR2-CE1
TYR2-CE2
TYR2-CZ
TYR2-OH
TYR2-H
TYR2-HA
TYR2-HB2
TYR2-HB3
TYR2-HD1
TYR2-HD2
TYR2-HE1
TYR2-HE2
TYR2-HH
ASP3-N
ASP3-CA
ASP3-C
ASP3-O
ASP3-CB
ASP3-CG
ASP3-OD1
ASP3-OD2
ASP3-H
ASP3-HA
ASP3-HB2
ASP3-HB3
PRO4-N
PRO4-CA
PRO4-C
PRO4-O
PRO4-CB
PRO4-CG
PRO4-CD
PRO4-HA
PRO4-HB2
PRO4-HB3
PRO4-HG2
PRO4-HG3
PRO4-HD2
PRO4-HD3
GLU5-N
GLU5-CA
GLU5-C
GLU5-O
GLU5-CB
GLU5-CG
GLU5-CD
GLU5-OE1
GLU5-OE2
GLU5-H
GLU5-HA
GLU5-HB2
GLU5-HB3
GLU5-HG2
GLU5-HG3
THR6-N
THR6-CA
THR6-C
THR6-O
THR6-CB
THR6-OG1
THR6-CG2
THR6-H
THR6-HA
THR6-HB
THR6-HG1
THR6-HG21
THR6-HG22
THR6-HG23
GLY7-N
GLY7-CA
GLY7-C
GLY7-O
GLY7-H
GLY7-HA2
GLY7-HA3
THR8-N
THR8-CA
THR8-C
THR8-O
THR8-CB
THR8-OG1
THR8-CG2
THR8-H
THR8-HA
THR8-HB
THR8-HG1
THR8-HG21
THR8-HG22
THR8-HG23
TRP9-N
TRP9-CA
TRP9-C
TRP9-O
TRP9-CB
TRP9-CG
TRP9-CD1
TRP9-CD2
TRP9-NE1
TRP9-CE2
TRP9-CE3
TRP9-CZ2
TRP9-CZ3
TRP9-CH2
TRP9-H
TRP9-HA
TRP

In [26]:
chain_index = 11
for residue in traj.topology.chain(chain_index).residues:
    for atom in residue.atoms:
        print(atom)

    break

G2-O5'

G2-C5'

G2-C4'

G2-O4'

G2-C3'

...

G2-H8

G2-H1

G2-H21

G2-H22

G2-HO5'

In [ ]:
def get_com_ssdna(traj: md.Trajectory, chain_index: int):
    """
    Gets the center of mass of the sugar, phosphate, base part from the traj and store them in dictionary.
    """

    for residue in traj.topology.chain(chain_index):
        print

In [22]:
[1,2,3] in [1,2,3,4]

False

In [23]:
def remove_hyrdogen(trajj: md.Trajectory) -> md.Trajectory:
    """
    Removes hydrogen atoms from a mdtraj.Trajectory object.
    
    Parameters
    ----------
    traj : mdtraj.Trajectory
        The input trajectory.
    
    Returns
    -------
    mdtraj.Trajectory
        The output trajectory containing only non-hydrogen atoms.
    """
    # Select all non-hydrogen atoms
    non_hydrogen_atoms = traj.topology.select("not element H")

    # Create a new trajectory containing only the non-hydrogen atoms
    traj_no_hydrogen = traj.atom_slice(non_hydrogen_atoms)

    return traj_no_hydrogen

In [24]:
traj = remove_hyrdogen(traj)

In [32]:
traj.top.select("chain")

ValueError: Cannot use a single literal as a boolean.

In [ ]:
[x.]